In [44]:
pip install openai

In [45]:
pip install --upgrade pip

Step 1 - Scraping your Linkedin profile

In [46]:
import os

# Set API keys for OpenAI and LinkedIn as environment variables
os.environ['OPENAI_API_KEY'] = "sk-proj-RbHcmwqUQ95Zk88TSbpYkuDWViONgTfXhf2HcJuBJAflTjXVzMtnHYSU1boLpeJVAdT3BlbkFJZS-lZdRRZhCiicxn_IyA6b99Gebekg1_QjfDLRHidw71fX88Ghpu9_yf_6vA_Vj5trGJGMA"
os.environ['LINKEDIN_API_KEY'] = "3bc8fa18msh858eb859e44239dp18d637a5351326"


In [47]:
# CRUCIAL STEP : Please a Linkedin profile here
profile_url = "https://www.linkedin.com/in/ali-cherifialaoui/" # You can also try with two different profiles (i.e https://linkedin.com/in/riad-hadjira or https://www.linkedin.com/in/ali-cherifialaoui/)

In [48]:
import requests
import openai
import os
import json
import pandas as pd
from IPython.display import display

# Retrieve API keys from environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")
linkedin_api_key = os.getenv("LINKEDIN_API_KEY")

# Ensure the API keys are correctly set
if not linkedin_api_key:
    print("Error: LinkedIn API key is missing. Please set it as an environment variable.")
    exit(1)

if not openai.api_key:
    print("Error: OpenAI API key is missing. Please set it as an environment variable.")
    exit(1)

In [49]:
# LinkedIn Scraper API setup
profile_scrape_url = "https://linkedin-data-api.p.rapidapi.com/get-profile-data-by-url"
headers = {
    "x-rapidapi-key": linkedin_api_key,
    "x-rapidapi-host": "linkedin-data-api.p.rapidapi.com"
}

# Function to scrape LinkedIn profile data
def scrape_profile(profile_url):
    querystring = {"url": profile_url}
    response = requests.get(profile_scrape_url, headers=headers, params=querystring)
    if response.status_code == 200:
        profile_data = response.json()
        return profile_data
    else:
        print(f"Error: Unable to scrape profile. Status Code: {response.status_code}, Response: {response.text}")
        return None


In [50]:
def analyze_profile(profile_data):
    if not profile_data:
        print("Error: No profile data provided for analysis.")
        return None

    profile_text = json.dumps(profile_data)  # Convert profile data to JSON string for analysis
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {
                    "role": "system",
                    "content": """
                    You are a career professional who helps students and young professionals find the most suitable job opportunities based on their skills, education, and experiences.
                    The context is that the person is currently a Master's (M2) student and will enter the job market in a few months or weeks. Therefore, do not recommend senior-level or highly experienced positions.
                    Your goal is to suggest entry-level or junior positions that align with the candidate's current background and aspirations.
                    """
                },
                {
                    "role": "user",
                    "content": f"""
                    Task: Analyze the following LinkedIn profile data to suggest:
                    1. A suitable job title (entry-level or junior)
                    2. A detailed reason for the suggestion
                    3. Key keywords for job search
                    4. Key insights from the profile, presented in a human-readable format (e.g., bullet points or paragraphs).

                    Format: The output should be in JSON format with the following fields:
                    - 'suggested_job_title': A single suggested job title
                    - 'reason': A detailed explanation of the suggestion
                    - 'keywords': A list of relevant keywords for job search
                    - 'profile_insights': Key insights, written in a clear and structured format suitable for human reading.

                    Here is the LinkedIn profile data to analyze:
                    {profile_text}
                    """
                }
            ]
        )

        # Extracting the response content
        suggestion = response['choices'][0]['message']['content'].strip()

        try:
            # Parse the output to JSON
            parsed_response = json.loads(suggestion)
            # Ensure the response contains all expected fields
            required_fields = ['suggested_job_title', 'reason', 'keywords', 'profile_insights']
            if all(field in parsed_response for field in required_fields):
                print("Analysis complete!")
                return parsed_response
            else:
                print("Error: Missing one or more required fields in the response. Full response:")
                print(parsed_response)
                return None
        except json.JSONDecodeError:
            print("Error: Malformed JSON received from the API. Raw response:")
            print(suggestion)
            return None

    except openai.error.InvalidRequestError as e:
        print("InvalidRequestError: Check the model or the input parameters.")
        print(e)
    except openai.error.AuthenticationError:
        print("AuthenticationError: Verify your API key.")
    except openai.error.RateLimitError:
        print("RateLimitError: You have exceeded your quota. Check your billing.")
    except openai.error.OpenAIError as e:
        print("OpenAIError: A general error occurred.")
        print(e)
    except Exception as e:
        print(f"Error: An unexpected error occurred - {e}")
        return None


In [51]:
# Function to create a dataset from the job suggestion and profile insights
def create_dataset(suggestion):
    if suggestion:
        # Create a DataFrame from the suggestion data
        data = {
            'Suggested Job Title': [suggestion.get('suggested_job_title')],
            'Reason': [suggestion.get('reason')],
            'Keywords for Search': [', '.join(suggestion.get('keywords', []))],
            'Profile Insights': [json.dumps(suggestion.get('profile_insights', {}))]
        }
        df = pd.DataFrame(data)
        return df
    else:
        print("No suggestion data available to create a dataset.")
        return None

In [52]:
# Function to display the dataset in a simple table
def display_dataset(df):
    if df is not None:
        print("\n--- Job Suggestion Dataset ---")
        display(df.style.set_properties(**{'text-align': 'left'}).set_table_styles(
            [{'selector': 'th', 'props': [('text-align', 'left')]}]
        ).set_caption("<b>Job Suggestion Overview</b>").set_table_attributes('border="1" class="dataframe table table-hover table-bordered"'))
    else:
        print("No dataset to display.")

In [53]:
# Full process to suggest a job and create a dataset based on LinkedIn profile
def main():
    profile_url
    profile_data = scrape_profile(profile_url)
    if profile_data:
        suggestion = analyze_profile(profile_data)
        dataset = create_dataset(suggestion)
        display_dataset(dataset)
        if dataset is not None:
            global job_suggestion_dataset
            job_suggestion_dataset = dataset  # Saving dataset to a global variable for further use

if __name__ == "__main__":
    main()

Analysis complete!

--- Job Suggestion Dataset ---


,Suggested Job Title,Reason,Keywords for Search,Profile Insights
0,Junior Data Analyst,"Based on the LinkedIn profile data provided, Ali seems to possess strong analytical skills and technical know-how as seen from his skills, certifications, and past positions. Ali was involved in data analysis and digital transformation. He has considerable experience using analytical tools like SQL, Python, Power BI, and has certifications in Data Science for Finance. As such, a role that involves data analysis and interpretation, like a Junior Data Analyst, would be suitable for him.","Data Analysis, Python, SQL, Power BI, Finance, Data Science, Digital Transformation, Strategy Consulting","[""Ali has a solid background in data analysis and digital transformation, having worked in related roles at companies such as Capgemini and EIGHT ADVISORY."", ""He has demonstrated proficiency in tools used for data analysis like SQL, Python, and Power BI."", ""He has a Masters in Data Science for Finance, thus understands the financial domain as well."", ""At EIGHT ADVISORY, he worked on financial data optimization and operational restructuring, indicating his ability to use analytical skills in practical, business-centric scenarios."", ""In addition to technical skills, Ali also possesses strong strategic skills and has worked directly with C-level executives, indicating his ability to communicate complex insights effectively.""]"


Step 2 : Matching jobs with your profile

In [58]:
# Job Search API Configuration
JOB_SEARCH_URL = "https://jobs-search-api.p.rapidapi.com/getjobs"
JOB_SEARCH_API_KEY = "2acddeaa1mshd153bfe9f1fp1b2c21f99d16a2"
job_search_headers = {
    "x-rapidapi-key": JOB_SEARCH_API_KEY,
    "x-rapidapi-host": "jobs-search-api.p.rapidapi.com",
    "Content-Type": "application/json"
}

# Function to search jobs using extracted data
def search_jobs(suggested_job_title, location, keywords):
    payload = {
        "search_term": suggested_job_title,
        "location": location,
        "results_wanted": 10,
        "site_name": ["linkedin", "glassdoor", "indeed"],
        "distance": 50,
        "job_type": "fulltime",
        "is_remote": False,
        "linkedin_fetch_description": False,
        "hours_old": 72
    }
    try:
        response = requests.post(JOB_SEARCH_URL, json=payload, headers=job_search_headers)
        if response.status_code == 200:
            job_data = response.json()
            return job_data.get("jobs", [])
        else:
            print(f"Error: Job search request failed. Status Code: {response.status_code}, Response: {response.text}")
            return []
    except Exception as e:
        print(f"Error during job search: {e}")
        return []

# Function to display job results
def display_jobs(job_results):
    if job_results:
        df_jobs = pd.DataFrame(job_results)
        print("\n--- Job Search Results ---")
        columns_to_display = ['title', 'company', 'location', 'job_url', 'date_posted']
        available_columns = [col for col in columns_to_display if col in df_jobs.columns]
        if available_columns:
            display(df_jobs[available_columns])
        else:
            display(df_jobs)
    else:
        print("No job results found.")

# Main function to integrate all steps
def main():
    # LinkedIn profile URL
    profile_url

    # Step 1: Scrape LinkedIn profile
    print("Scraping LinkedIn profile...")
    profile_data = scrape_profile(profile_url)
    if not profile_data:
        print("Error: Failed to scrape profile.")
        return

    # Step 2: Analyze the profile
    print("Analyzing LinkedIn profile...")
    suggestion = analyze_profile(profile_data)
    if not suggestion:
        print("Error: Failed to analyze profile.")
        return

    # Step 3: Use the suggestion to search for jobs
    print("\nSearching for jobs based on profile analysis...")
    job_results = search_jobs(
        suggested_job_title=suggestion["suggested_job_title"],
        location=profile_data.get("location", "France"),  # Use location based on your interests
        keywords=suggestion["keywords"]
    )

    # Step 4: Display job results
    display_jobs(job_results)

if __name__ == "__main__":
    main()


Scraping LinkedIn profile...
Analyzing LinkedIn profile...
Analysis complete!

Searching for jobs based on profile analysis...

--- Job Search Results ---


,title,company,location,job_url,date_posted
0,Data Analyst H/F,emeis,"Puteaux, Île-de-France, France",https://www.linkedin.com/jobs/view/4087228308,2024-11-28
1,Data Analyst F/H,SOFTEAM,"Paris, Île-de-France, France",https://www.linkedin.com/jobs/view/4044551299,2024-11-28
2,Data Analyst - Intern (Remote Internship - Dat...,Kreativstorm,"Lassay-sur-Croisne, Centre-Val de Loire, France",https://www.linkedin.com/jobs/view/4087221059,2024-11-28
3,Data Analyst (All Genders),Dailymotion,"Paris, Île-de-France, France",https://www.linkedin.com/jobs/view/4084209312,2024-11-27
4,Data Analyst,Semji,"Lyon, Auvergne-Rhône-Alpes, France",https://www.linkedin.com/jobs/view/4086446436,2024-11-27
5,Data Analyst F/H,CGI,"Bordeaux, Nouvelle-Aquitaine, France",https://www.linkedin.com/jobs/view/4084788991,
6,Data Analyste Power BI - junior(e) ou expérime...,SEGULA Technologies,"Courbevoie, Île-de-France, France",https://www.linkedin.com/jobs/view/4084703601,
7,Data Analyst H/F,Wewyse,"Paris, Île-de-France, France",https://www.linkedin.com/jobs/view/4077660528,
8,Data Analyst - LaunchOps,Voodoo,"Paris, Île-de-France, France",https://www.linkedin.com/jobs/view/4085010273,
9,Data Analyst,Swan,"Paris, Île-de-France, France",https://www.linkedin.com/jobs/view/4086202834,


Step 3 - Find the average salary depending on your location and preferences

In [55]:
import requests
import pandas as pd

# Glassdoor Salary API Configuration
SALARY_API_URL = "https://job-salary-data.p.rapidapi.com/job-salary"
SALARY_API_KEY = "2acddeaamsh9836d153bfe9f1fp17b81f99d16a2"
salary_headers = {
    "x-rapidapi-key": SALARY_API_KEY,
    "x-rapidapi-host": "job-salary-data.p.rapidapi.com"
}

# Function to fetch salary data
def get_salary_data(job_title, location):
    """
    Fetch salary data for a specific job title and location.

    Args:
        job_title (str): The job title to search for.
        location (str): The location to search in.

    Returns:
        dict: Salary details or an error message.
    """
    querystring = {
        "job_title": job_title,
        "location": location,
        "location_type": "ANY",
        "years_of_experience": "ALL"
    }
    print("Payload sent to Salary API:", querystring)  # Debug: Show payload

    try:
        response = requests.get(SALARY_API_URL, headers=salary_headers, params=querystring)
        print("Status Code:", response.status_code)  # Debug: Show status code
        print("Full Response Text:", response.text)  # Debug: Show full response

        if response.status_code == 200:
            salary_data = response.json()
            if salary_data.get("status") == "OK" and "data" in salary_data:
                return salary_data["data"][0]  # Return the first set of salary data
            else:
                print("Error: No salary data found for the given parameters.")
                return {}
        else:
            print(f"Error: Salary API request failed. Status Code: {response.status_code}")
            return {}
    except Exception as e:
        print(f"Error during salary data retrieval: {e}")
        return {}

# Function to display salary data
def display_salary_data(salary_data):
    """
    Display salary data in a readable format.

    Args:
        salary_data (dict): Salary details from the API.
    """
    if salary_data:
        print("\n--- Salary Data ---")
        print(f"Job Title: {salary_data.get('job_title', 'N/A')}")
        print(f"Location: {salary_data.get('location', 'N/A')}")
        print(f"Min Salary: {round(salary_data.get('min_salary', 0))} {salary_data.get('salary_currency', 'N/A')}")
        print(f"Max Salary: {round(salary_data.get('max_salary', 0))} {salary_data.get('salary_currency', 'N/A')}")
        print(f"Median Salary: {round(salary_data.get('median_salary', 0))} {salary_data.get('salary_currency', 'N/A')}")
        print(f"Salary Period: {salary_data.get('salary_period', 'N/A')}")
        print(f"Data Source: {salary_data.get('publisher_name', 'N/A')} ({salary_data.get('publisher_link', 'N/A')})")
        print(f"Confidence: {salary_data.get('confidence', 'N/A')}")
    else:
        print("No salary data available.")

# Main function to retrieve and display salary data
def main():
    # Example input
    job_title = "Data Analyst" # You can type the suggested job that our solution recommended to you, or even another job to compare
    location = "France" # You can choose the country where you would love to work, or even others if you want to compare

    # Fetch salary data
    salary_data = get_salary_data(job_title, location)

    # Display salary data
    display_salary_data(salary_data)

if __name__ == "__main__":
    main()


Payload sent to Salary API: {'job_title': 'Data Analyst', 'location': 'France', 'location_type': 'ANY', 'years_of_experience': 'ALL'}
Status Code: 200
Full Response Text: {"status":"OK","request_id":"91c05fc8-324e-4933-abe8-7b50e80b2d4c","data":[{"location":"France","job_title":"Data Analyst","min_salary":40417.5,"max_salary":56000,"median_salary":45000,"salary_period":"YEAR","salary_currency":"EUR","publisher_name":"Glassdoor","publisher_link":"https://www.glassdoor.com/Salaries/company-salaries.htm?suggestCount=0&suggestChosen=false&sc.keyword=Data%20Analyst&locT=N&locId=86","confidence":"VERY_HIGH"}]}

--- Salary Data ---
Job Title: Data Analyst
Location: France
Min Salary: 40418 EUR
Max Salary: 56000 EUR
Median Salary: 45000 EUR
Salary Period: YEAR
Data Source: Glassdoor (https://www.glassdoor.com/Salaries/company-salaries.htm?suggestCount=0&suggestChosen=false&sc.keyword=Data%20Analyst&locT=N&locId=86)
Confidence: VERY_HIGH
